# Face Detection

In [1]:
from __future__ import print_function
import cv2

class FaceDetector:
    def __init__(self, faceCascadePath):
        self.faceCascade = cv2.CascadeClassifier(faceCascadePath)
        
    def detect(self, image, scaleFactor = 1.1, minNeighbors = 5, minSize = (30, 30)):
        rects = self.faceCascade.detectMultiScale(image, scaleFactor = scaleFactor, minNeighbors =minNeighbors, minSize = minSize, flags = cv2.CASCADE_SCALE_IMAGE)
        return rects

image_location = './images/obama.png'
image = cv2.imread(image_location)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

fd = FaceDetector("./cascades/haarcascade_frontalface_default.xml")
faceRects = fd.detect(gray, scaleFactor = 1.1, minNeighbors = 5, minSize = (30, 30))
print("I found {} face(s)".format(len(faceRects)))

for (x, y, w, h) in faceRects:
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
    
cv2.imshow("Faces", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

I found 1 face(s)


# Webcam Face Detection

In [2]:
from __future__ import print_function
import cv2

class FaceDetector:
    def __init__(self, faceCascadePath):
        self.faceCascade = cv2.CascadeClassifier(faceCascadePath)
        
    def detect(self, image, scaleFactor = 1.1, minNeighbors = 5, minSize = (30, 30)):
        rects = self.faceCascade.detectMultiScale(image, scaleFactor = scaleFactor, minNeighbors =minNeighbors, minSize = minSize, flags = cv2.CASCADE_SCALE_IMAGE)
        return rects

def resize(image, width = None, height = None, inter = cv2.INTER_AREA):
    dim = None
    (h, w) = image.shape[:2]
    if width is None and height is None:
        return image
    if width is None:
        r = height / float(h)
        dim = (int(w *r), height)
    else:
        r = width / float(w)
        dim = (width, int(h *r))
    resized = cv2.resize(image, dim, interpolation = inter)
    return resized

fd = FaceDetector("./cascades/haarcascade_frontalface_default.xml")
camera = cv2.VideoCapture(0)

while True:
    (grabbed, frame) = camera.read()
    if not grabbed:
        break
    frame = resize(frame, width = 300)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faceRects = fd.detect(gray, scaleFactor = 1.1, minNeighbors = 5, minSize = (30, 30))
    #print("I found {} face(s)".format(len(faceRects)))
    frameClone = frame.copy()
    
    for (fX, fY, fW, fH) in faceRects:
        cv2.rectangle(frameClone, (fX, fY), (fX + fW, fY + fH), (0, 255, 0), 2)
        
    cv2.imshow("Face", frameClone)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
        
camera.release()
cv2.destroyAllWindows()
    

# Object Tracking in Video

In [3]:
import numpy as np
import cv2
import time
    
blueLower = np.array([100, 67, 0], dtype = "uint8")
blueUpper = np.array([255, 128, 50], dtype = "uint8")

camera = cv2.VideoCapture('./video/iphonecase.mov')

while True:
    (grabbed, frame) = camera.read()
    
    if not grabbed:
        break
        
    blue = cv2.inRange(frame, blueLower, blueUpper)
    blue = cv2.GaussianBlur(blue, (3, 3), 0)
    (_, cnts, _) = cv2.findContours(blue.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if len(cnts) > 0:
        cnt = sorted(cnts, key = cv2.contourArea, reverse = True)[0]# find the largest contour
        
        rect = np.int32(cv2.boxPoints(cv2.minAreaRect(cnt)))
        cv2.drawContours(frame, [rect], -1, (0, 255, 0), 2)
        
    cv2.imshow("Tracking", frame)
    cv2.imshow("Binary", blue)
    
    time.sleep(0.025)
    
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
    
camera.release()
cv2.destroyAllWindows()


# Eye Tracking

In [4]:
import cv2

class EyeTracker:
    def __init__(self, faceCascadePath, eyeCascadePath):
        self.faceCascade = cv2.CascadeClassifier(faceCascadePath)
        self.eyeCascade = cv2.CascadeClassifier(eyeCascadePath)
        
    def track(self, image):
        faceRects = self.faceCascade.detectMultiScale(image, scaleFactor = 1.1, minNeighbors = 5, minSize = (30, 30), flags = cv2.CASCADE_SCALE_IMAGE)
        rects = []
        for (fX, fY, fW, fH) in faceRects:
            faceROI = image[fY:fY + fH, fX:fX + fW]
            rects.append((fX, fY, fX + fW, fY + fH))
            
            eyeRects = self.eyeCascade.detectMultiScale(faceROI, scaleFactor = 1.1, minNeighbors = 10, minSize = (20, 20), flags = cv2.CASCADE_SCALE_IMAGE)
            
            for (eX, eY, eW, eH) in eyeRects:
                rects.append((fX + eX, fY + eY, fX + eX + eW, fY + eY + eH))
        return rects
    
et = EyeTracker("./cascades/haarcascade_frontalface_default.xml", "./cascades/haarcascade_eye.xml")
camera = cv2.VideoCapture(0)

def resize(image, width = None, height = None, inter = cv2.INTER_AREA):
    dim = None
    (h, w) = image.shape[:2]
    if width is None and height is None:
        return image
    if width is None:
        r = height / float(h)
        dim = (int(w *r), height)
    else:
        r = width / float(w)
        dim = (width, int(h *r))
    resized = cv2.resize(image, dim, interpolation = inter)
    return resized

while True:
    (grabbed, frame) = camera.read()
    
    if not grabbed:
        break
    frame = resize(frame, width = 300)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    rects = et.track(gray)
    
    for rect in rects:
        cv2.rectangle(frame, (rect[0], rect[1]), (rect[2], rect[3]), (0, 255, 0), 2)
        
    cv2.imshow("Tracking", frame)
    
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

camera.release()
cv2.destroyAllWindows()
        

# Handwriting Recognition with Hog